## Neural network for detection of AF peaks


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset

import torchvision
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

import math
import time
import pandas as pd
import numpy as np
import os

In [2]:
# Loading the dataset into a dataframe
path = 'Preprocessed_AFData.xlsx' #Path
df = pd.read_excel(path) #Turning into a dataframe
print('shape of the dataframe:', np.shape(df))
df.head() #Printing an example of the dataframe

shape of the dataframe: (150000, 31)


,data1,data2,data3,data4,data5,data6,data7,data8,data9,data10,...,data22,data23,data24,data25,data26,data27,data28,data29,data30,Control
0,0.0,0.0,0.0,0.1,-0.1,0.0,0.0,0.4,0.100000,0.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,1.0,-0.4,-0.6,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.954545,0.045455,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [3]:
# Splitting the data set into training and test set
train_set, rest_set = train_test_split(df, test_size=0.4, random_state=100) #Splitting into training and test part
test_set, val_set = train_test_split(rest_set, test_size=0.5, random_state=42)

#Separating signal from label
X_train = train_set.iloc[:,:30].to_numpy() 
y_train = train_set.iloc[:,30].to_numpy()
X_test = test_set.iloc[:,:30].to_numpy()
y_test = test_set.iloc[:,30].to_numpy()
X_val = val_set.iloc[:,:30].to_numpy()
y_val = val_set.iloc[:,30].to_numpy()

#Normalize the training data
# scaler = StandardScaler() 
# print(X_train[1,:])
# X_train = scaler.fit_transform(X_train)
# print(X_train[1,:])
# X_test = scaler.transform(X_test)

print('shape of the train dataframe:', np.shape(X_train))
print('shape of the train label dataframe:', np.shape(y_train))
print('shape of the test dataframe:', np.shape(X_test))


shape of the train dataframe: (90000, 30)
shape of the train label dataframe: (90000,)
shape of the test dataframe: (30000, 30)


In [12]:
# Creating DataLoader instances
batch_size = 512

#Creating a Dataset instance for the dataloader to call
class dataset(Dataset):
    def __init__(self,x,y):
        self.x = torch.tensor(x,dtype=torch.float32)
        self.y = torch.tensor(y,dtype=torch.float32)
        self.length = self.x.shape[0]

    def __getitem__(self,idx):
        return self.x[idx],self.y[idx]
    
    def __len__(self):
        return self.length

X_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(y_train)
X_test = torch.FloatTensor(X_test)
y_test = torch.FloatTensor(y_test)
X_val = torch.FloatTensor(X_val)
y_val = torch.FloatTensor(y_val)

train_set = dataset(X_train, y_train) #Training data set
test_set = dataset(X_test, y_test) #Test data set
val_set = dataset(X_val, y_val) #Validation data set

train_loader = torch.utils.data.DataLoader(train_set,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(test_set,
                                         batch_size=1,
                                         shuffle=False)

val_loader = torch.utils.data.DataLoader(val_set,
                                        batch_size=1,
                                        shuffle=False)

C:\Users\bas_w\Documents\PYTHON\Anaconda\envs\deeplearn_course\lib\site-packages\ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
C:\Users\bas_w\Documents\PYTHON\Anaconda\envs\deeplearn_course\lib\site-packages\ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [7]:
# Define the network
class NeuralNet(nn.Module):

    def __init__(self, in_channels):
        super().__init__()
        self.fc1 = nn.Linear(in_channels, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64 , 1)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):  
        x = self.fc1(x)
        x = self.relu(x)
        x = self.batchnorm1(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

In [30]:
# Setting some parameters
lr = 0.001
epochs = 200
model = NeuralNet(in_channels = X_train.shape[1])
optimizer = torch.optim.Adam(model.parameters(),lr=lr)
# criterion = nn.BCEWithLogitsLoss() # No weights
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.FloatTensor ([20 / 1])) # With weights
save_path = 'Checkpoints/Saved_model_weights_with_class_weights20'


In [31]:
# Accuracy statistic
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [32]:
# Training the network
start = time.time()
accuracy_best = -1
best_loss = 99999
train_loss = []
train_acc = []
test_acc = []

for i in range(1, epochs+1):
    #Training the network
    model.train()
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    
    #Evaluating the network
    model.eval()
    correct = 0
    test_epoch_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            y_test_pred = model(X_batch)
            test_loss = criterion(y_test_pred, y_batch.unsqueeze(1))
            y_test_pred = torch.sigmoid(y_test_pred)
            y_pred_tag = torch.round(y_test_pred).item()
            test_epoch_loss += test_loss.item()
            if y_pred_tag == y_batch.item():
                correct += 1
    
    print(correct)       
    accuracy = 100 * correct / len(test_set)
    print(f'Epoch {i+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')
    print(f'Epoch {i+0:03}: | Test Loss: {test_epoch_loss/len(test_loader)} | Test Acc: {accuracy:.3f}')
    if test_epoch_loss < best_loss:
        accuracy_best = accuracy
        best_loss = test_epoch_loss
        torch.save(model.state_dict(), save_path)
        print('New model saved!!')
        
    train_loss.append(epoch_loss/len(train_loader))
    train_acc.append(epoch_acc/len(train_loader))
    test_acc.append(accuracy)


7388
Epoch 001: | Loss: 2.25152 | Acc: 24.080
Epoch 001: | Test Loss: 2.2104756892184416 | Test Acc: 24.627
New model saved!!
7389
Epoch 002: | Loss: 2.17245 | Acc: 24.068
Epoch 002: | Test Loss: 2.1966433256347973 | Test Acc: 24.630
New model saved!!
7390
Epoch 003: | Loss: 2.15833 | Acc: 24.097
Epoch 003: | Test Loss: 2.1870778719703354 | Test Acc: 24.633
New model saved!!
7393
Epoch 004: | Loss: 2.15406 | Acc: 24.074
Epoch 004: | Test Loss: 2.185498139665524 | Test Acc: 24.643
New model saved!!
7398
Epoch 005: | Loss: 2.15094 | Acc: 24.114
Epoch 005: | Test Loss: 2.1832016595502695 | Test Acc: 24.660
New model saved!!
7416
Epoch 006: | Loss: 2.14902 | Acc: 24.159
Epoch 006: | Test Loss: 2.180920368085305 | Test Acc: 24.720
New model saved!!
7420
Epoch 007: | Loss: 2.14788 | Acc: 24.199
Epoch 007: | Test Loss: 2.180075200297435 | Test Acc: 24.733
New model saved!!
7475
Epoch 008: | Loss: 2.14728 | Acc: 24.284
Epoch 008: | Test Loss: 2.1811959581454596 | Test Acc: 24.917
7497
Epoch 00

16546
Epoch 075: | Loss: 2.13914 | Acc: 56.682
Epoch 075: | Test Loss: 2.178442023080587 | Test Acc: 55.153
19034
Epoch 076: | Loss: 2.13899 | Acc: 58.455
Epoch 076: | Test Loss: 2.176587982648611 | Test Acc: 63.447
19141
Epoch 077: | Loss: 2.13830 | Acc: 57.330
Epoch 077: | Test Loss: 2.179162794651588 | Test Acc: 63.803
20123
Epoch 078: | Loss: 2.13924 | Acc: 57.903
Epoch 078: | Test Loss: 2.1757369317869344 | Test Acc: 67.077
18862
Epoch 079: | Loss: 2.13913 | Acc: 58.824
Epoch 079: | Test Loss: 2.1760750735998156 | Test Acc: 62.873
15021
Epoch 080: | Loss: 2.14032 | Acc: 53.716
Epoch 080: | Test Loss: 2.1779377793391546 | Test Acc: 50.070
16054
Epoch 081: | Loss: 2.13902 | Acc: 52.597
Epoch 081: | Test Loss: 2.180220209413767 | Test Acc: 53.513
18927
Epoch 082: | Loss: 2.13886 | Acc: 56.523
Epoch 082: | Test Loss: 2.175904414665699 | Test Acc: 63.090
19745
Epoch 083: | Loss: 2.13796 | Acc: 58.562
Epoch 083: | Test Loss: 2.1799532684286436 | Test Acc: 65.817
17079
Epoch 084: | Loss:

18251
Epoch 151: | Loss: 2.13663 | Acc: 59.886
Epoch 151: | Test Loss: 2.1807381723622483 | Test Acc: 60.837
18067
Epoch 152: | Loss: 2.13656 | Acc: 60.540
Epoch 152: | Test Loss: 2.178376179675261 | Test Acc: 60.223
19248
Epoch 153: | Loss: 2.13789 | Acc: 59.830
Epoch 153: | Test Loss: 2.1780355472822985 | Test Acc: 64.160
19146
Epoch 154: | Loss: 2.13705 | Acc: 59.307
Epoch 154: | Test Loss: 2.1776465440134207 | Test Acc: 63.820
17705
Epoch 155: | Loss: 2.13802 | Acc: 58.398
Epoch 155: | Test Loss: 2.1790512676695983 | Test Acc: 59.017
18067
Epoch 156: | Loss: 2.13786 | Acc: 59.591
Epoch 156: | Test Loss: 2.176260196371873 | Test Acc: 60.223
17648
Epoch 157: | Loss: 2.13641 | Acc: 60.170
Epoch 157: | Test Loss: 2.181990771694978 | Test Acc: 58.827
18091
Epoch 158: | Loss: 2.13750 | Acc: 58.699
Epoch 158: | Test Loss: 2.181221914591392 | Test Acc: 60.303
17953
Epoch 159: | Loss: 2.13714 | Acc: 58.881
Epoch 159: | Test Loss: 2.1789455868800482 | Test Acc: 59.843
18614
Epoch 160: | Loss

In [33]:
# Making predictions on validation set
y_pred_list = []
y_true_list = []
model.load_state_dict(torch.load(save_path))
model.eval()
correct = 0
with torch.no_grad():
    for X_batch, y_batch in val_loader:
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_true_list.append(y_batch.item())
        if y_pred_tag == y_batch.item():
            correct += 1
            
    accuracy = 100 * correct / len(test_set)
    print(correct)
    print(accuracy)

y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

16155
53.85


In [34]:
target_names = ['Label: 0','Label: 1']
print(classification_report(y_true_list, y_pred_list,target_names=target_names))

              precision    recall  f1-score   support

    Label: 0       0.99      0.39      0.56     22545
    Label: 1       0.35      0.98      0.51      7455

    accuracy                           0.54     30000
   macro avg       0.67      0.69      0.54     30000
weighted avg       0.83      0.54      0.55     30000

